In [8]:
import requests
import time
import datetime

In [9]:
import pandas as pd


In [10]:
def sendMessage(para, mensaje):
    url = 'http://localhost:3001/lead'
    
    data = {
        "message": mensaje,
        "phone": para
    }
    headers = {
        'Content-Type':'application/json'
    }
    print(data)
    response = requests.post(url, json=data, headers=headers)
    time.sleep(60)
    return response

In [11]:
now = datetime.datetime.now()

In [12]:
mario = '59175380725'
me = '59178194371'

In [13]:
def calcular_datos():
    df = pd.read_excel(r'C:/Documents/Ingenio Azucarero Guabira S.A/COOR_GERENCIA_CANA - Parte_Horarios/ReportePlaya.xlsx')
    filtro = df[(df['status'] == 'PL') | (df['status'] == 'IN')]
    cantidad_paquetes = filtro['cantPqt'].sum()
    cana_disponible = cantidad_paquetes * 14
    return (cantidad_paquetes, cana_disponible)

In [ ]:
while True:
    #now = datetime.datetime.now()
    cantidad_paquetes, cana_disponible = calcular_datos()
    mensaje = f'''
*Cantidad de paquetes en playa:* {cantidad_paquetes} paquetes.
*Cantidad de caña disponible:* {cana_disponible} toneladas.
'''
    sendMessage(me, mensaje)

{'message': '\n*Cantidad de paquetes en playa:* 102 paquetes.\n*Cantidad de caña disponible:* 1428 toneladas.\n', 'phone': '59178194371'}
{'message': '\n*Cantidad de paquetes en playa:* 102 paquetes.\n*Cantidad de caña disponible:* 1428 toneladas.\n', 'phone': '59178194371'}
{'message': '\n*Cantidad de paquetes en playa:* 102 paquetes.\n*Cantidad de caña disponible:* 1428 toneladas.\n', 'phone': '59178194371'}
